# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [33]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [34]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [35]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68584 entries, 0 to 68583
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CustomerID   68584 non-null  int64 
 1   FirstName    68584 non-null  object
 2   LastName     68584 non-null  object
 3   SalesID      68584 non-null  int64 
 4   ProductID    68584 non-null  int64 
 5   ProductName  68584 non-null  object
 6   Quantity     68584 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 3.7+ MB


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [37]:
q = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()
q

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [38]:
q_matrix = pd.pivot_table(q, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0)

q_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [39]:
PDIST = pdist(q_matrix.T, metric='euclidean')

customer_similarity_matrix = 1 / (1 + squareform(PDIST))

customer_similarity_matrix


array([[1.        , 0.0774211 , 0.08704668, ..., 0.00435486, 0.00416667,
        0.00433325],
       [0.0774211 , 1.        , 0.07844774, ..., 0.0043666 , 0.00416601,
        0.00433456],
       [0.08704668, 0.07844774, 1.        , ..., 0.00436527, 0.00417944,
        0.00433325],
       ...,
       [0.00435486, 0.0043666 , 0.00436527, ..., 1.        , 0.0032772 ,
        0.00326622],
       [0.00416667, 0.00416601, 0.00417944, ..., 0.0032772 , 1.        ,
        0.0032883 ],
       [0.00433325, 0.00433456, 0.00433325, ..., 0.00326622, 0.0032883 ,
        1.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [29]:
customer_similarity_df = pd.DataFrame(customer_similarity_matrix, index=q_matrix.columns, columns=q_matrix.columns)


In [30]:
def get_top_similar_customers(customer_id, customer_similarity_df, num_similar_customers=5):
    similar_customers = customer_similarity_df[customer_id]

    similar_customers = similar_customers[similar_customers.index != customer_id]

    top_similar_customers = similar_customers.sort_values(ascending=False).head(num_similar_customers)

    return top_similar_customers.index.tolist()


In [31]:
chosen_customer_id = 61288

# Get the top 5 most similar customers for the chosen CustomerID
similar_customers_list = get_top_similar_customers(chosen_customer_id, customer_similarity_df)


In [32]:
print(f"Top 5 most similar customers for CustomerID {chosen_customer_id}: {similar_customers_list}")


Top 5 most similar customers for CustomerID 61288: [15166, 5968, 8539, 5224, 5104]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [40]:
similar_customers_records = q[q['CustomerID'].isin(similar_customers_list)]

In [41]:
similar_customers_records

,CustomerID,ProductName,Quantity
3570,5104,Apricots - Halves,2
3571,5104,Bacardi Breezer - Tropical,2
3572,5104,Bay Leaf,2
3573,5104,"Beans - Kidney, Canned",2
3574,5104,Beans - Wax,2
...,...,...,...
10120,15166,"Wine - Magnotta, Merlot Sr Vqa",4
10121,15166,"Wine - Red, Harrow Estates, Cab",4
10122,15166,Wine - Redchard Merritt,4
10123,15166,Wine - Ruffino Chianti,4


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [43]:
similar_customers_aggregated = similar_customers_records.groupby('ProductName')['Quantity'].sum().reset_index()
similar_customers_ranked = similar_customers_aggregated.sort_values(by='Quantity', ascending=False)

similar_customers_ranked

,ProductName,Quantity
19,Beef - Rib Eye Aaa,12
100,Jagermeister,12
54,Chicken - Soup Base,12
113,Lime Cordial - Roses,10
108,"Lamb - Pieces, Diced",10
...,...,...
161,"Salsify, Organic",2
160,Salmon Steak - Cohoe 8 Oz,2
159,Salmon - Sockeye Raw,2
158,"Salmon - Atlantic, Skin On",2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.